~~测试结果反应，无论拼接多少其他同分类股票，对于准确度的提升是不大的，几乎没有。~~


## 测试内容

根据证监会的行业分类，取出同行业下所有股票（上市时间早于 2015-01-01,并且是A股上市）。

随机取出10条与待测试的股票组合，进行测试。（保留指数数据-`Wrapper_default`/`Normalize`）

测试参数：

* 窗口期：3
* 测试期：1

In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from LSTM_for_Stock.data_processor import Wrapper_default
from LSTM_for_Stock.data_processor import Normalize
from LSTM_for_Stock.data_processor import get_block_code
from LSTM_for_Stock.data_processor import get_ipo_date
import jupyter_helper
from keras.backend import clear_session
import pandas as pd
import numpy as np
import QUANTAXIS as QA
import datetime
import random

Using TensorFlow backend.


In [2]:
report={}
report['kind']=[]
report['acc']=[]
report['mae']=[]
report['loss']=[]
window=3
days=1

In [3]:
def test_code(code):
    return code[0] in ['0', '3', '6']

In [7]:
end=datetime.datetime(2005,1, 1)
codes = [code for code in get_block_code('000002') if test_code(code) and get_ipo_date(code) is not None and get_ipo_date(code)<=end]

QUANTAXIS>> 'code'
QUANTAXIS>> 'code'


In [8]:
len(codes),len(get_block_code('000002'))

(129, 151)

In [ ]:
for i in range(len(codes)*100):
    cs = random.sample(codes, 10)
    print('{0}/{1}'.format(i+1, len(codes)*100))
    print(','.join(cs))
    h = jupyter_helper.do(
        window=window,
        days=days,
        wrapper=Wrapper_default(),
        norm=Normalize(),
        appends=cs,
        summary=False)
    s = jupyter_helper.show_history(h, show_plt=False)
    report['kind'].append(','.join(cs))
    report['acc'].append(s['score'][2])
    report['mae'].append(s['score'][1])
    report['loss'].append(s['score'][0])

1/12900
600759,000502,600215,000897,000671,600576,000889,000042,000638,600767


## 报告

In [ ]:
pd.DataFrame(report,columns=['kind','acc','mae','loss']).set_index('kind').sort_values('acc',ascending=False).head()